In [1]:
from rnamodif.architectures.rodan_pretrained_modcaller import RodanPretrainedModcaller
from rnamodif.data_utils.dataloading2 import nanopore_datamodule
from rnamodif.data_utils.split_methods import get_kfold_splits, get_fullvalid_split, get_valid_portions
import pytorch_lightning as pl
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.callbacks import ModelCheckpoint

model = RodanPretrainedModcaller(lr=1e-4, use_Ns_for_loss=False, weighted_loss=False, use_basecalling_loss=False, loss_only_from_As=False, warmup_steps=1000)
dm = nanopore_datamodule(
    splits = get_kfold_splits(
        pos_exps=['m6A_33_covid'],
        neg_exps=['m6A_0_covid'], 
        total_k=5, 
        current_k=0
    )
    +get_fullvalid_split()(pos_files=['m6a_novoa_1', 'm6a_novoa_2', 'm6A_5_covid','m6A_10_covid'], neg_files=['UNM_novoa_1', 'UNM_novoa_2', 'UNM_novoa_short']),
    verbose=1, 
    batch_size=64, 
    valid_limit=1000, 
    workers=16,
    window=4096,
    normalization='rodan',
)

experiment_name = 'm6a_0v33_leaktest'
checkpoint_callback = ModelCheckpoint(
    dirpath=f"/home/jovyan/RNAModif/rnamodif/checkpoints_pl/{experiment_name}", 
    save_top_k=2, 
    monitor="valid_loss", 
    save_last=True, 
    save_weights_only=False
)

logger = CometLogger(api_key="TEVQbgxxvilM1WdTyqZLJ57ac", project_name='RNAModif', experiment_name=experiment_name) 
trainer= pl.Trainer(
    max_steps = 1000000, logger=logger, accelerator='gpu',
    auto_lr_find=False, val_check_interval=500,  
    log_every_n_steps=500, benchmark=True, precision=16,#)#,
    callbacks=[checkpoint_callback])


trainer.fit(model, dm)

CometLogger will be initialized in online mode
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory /home/jovyan/RNAModif/rnamodif/checkpoints_pl/m6a_0v33_leaktest exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type                  | Params
----------------------------------------------------------
0 | trainable_rodan | network               | 10.7 M
1 | original_rodan  | network               | 10.7 M
2 | mod_neuron      | Linear                | 769   
3 | acc             | BinaryAccuracy        | 0     
4 | cm              | BinaryConfusionMatrix | 0     
5 | auroc

Sanity Checking: 0it [00:00, ?it/s]

/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:219: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:86: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 64. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Training: 0it [00:00, ?it/s]

/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:657: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
